# Circuit autointerpretability

This stuff just sets up everything we need.

In [1]:
from autointerpretability import *

config = yaml.safe_load(open("config.yaml"))
llm_client = AzureOpenAI(
    azure_endpoint=config["base_url"],
    api_key=config["azure_api_key"],
    api_version=config["api_version"],
)

model = HookedTransformer.from_pretrained('gpt2-small')

dataset = load_dataset('Skylion007/openwebtext', split='train', streaming=True)
dataset = dataset.shuffle(seed=42, buffer_size=10_000)
tokenized_owt = tokenize_and_concatenate(dataset, model.tokenizer, max_length=128, streaming=True)
tokenized_owt = tokenized_owt.shuffle(42)
tokenized_owt = tokenized_owt.take(12800 * 2)
owt_tokens = np.stack([x['tokens'] for x in tokenized_owt])
owt_tokens_torch = torch.tensor(owt_tokens)

device = 'cpu'
tl_model, z_saes, transcoders = get_model_encoders(device=device)

/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer


/Users/charlesoneill/miniconda3/envs/anu/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Token indices sequence length is longer than the specified maximum sequence length for this model (73252 > 1024). Running this sequence through 

Loaded pretrained model gpt2-small into HookedTransformer

Loading SAEs...


100%|██████████| 12/12 [00:07<00:00,  1.58it/s]



Loading Transcoders...


100%|██████████| 12/12 [00:04<00:00,  2.62it/s]


Note you can specify the features you want to examine, in each layer, and just pass in either the relevant ZSAE or MLP transcoder depending on what component you want to look at. The `get_feature_scores` function will handle the differences. Let's have a look at the max-activating examples on Danny's features he wanted to check out (note you can slice `owt_tokens_torch` to run for shorter).

In [2]:
features = [16513, 7861]
sae = z_saes[8]
feature_scores = get_feature_scores(model, sae, owt_tokens_torch, features, batch_size=64)

ZSAE


100%|██████████| 400/400 [04:19<00:00,  1.54it/s]


Our feature scores are a tensor of shape `(batch, feature, seq_pos)`, and so I've got a function to help extract the max-activating examples for each feature. You need to specify the feature index, which is why it's helpful to know from above the features in your list.

In [3]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, 0, :], owt_tokens_torch, k=10, show_score=True)

Then, you can just pass it off to GPT-4 to interpret what's going on. Note that I haven't got access to `GPT-4o` with my credits yet, so this will have to wait a few days.

In [4]:
feature_interpretation = get_response(llm_client, examples_clean_text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
feature_interpretation

'This neuron is highly activated by specific mentions of a context or event happening at a certain place or location.'

However, instead of passing in individual features for specific components in specific layers, I created an object called `CircuitPrediction` to basically store all this stuff for you. I'll quickly illustrate how to use it in conjunction with the above.

In [6]:
cp = get_circuit_prediction(task='ioi', N=50)

100%|██████████| 50/50 [01:52<00:00,  2.25s/it]


The main thing you'll want to do with this is get features from certain components to look at on a specific task. The features for each component are stored in the circuit hypergraph. For instance:

In [8]:
cp.circuit_hypergraph

{'L0_H0': {'freq': 0.0, 'features': []},
 'L0_H1': {'freq': 0.8199999928474426,
  'features': [451,
   -1,
   20191,
   21082,
   18627,
   13846,
   451,
   2470,
   -1,
   14731,
   -1,
   18627,
   451,
   -1,
   3949,
   5142,
   -1,
   2680,
   13846,
   14731,
   451,
   -1,
   21082,
   18627,
   -1,
   451,
   3949,
   23825,
   18627,
   3949,
   17242,
   451,
   -1,
   3949,
   -1,
   10072,
   4229,
   18627,
   451,
   -1,
   10072,
   451,
   21082,
   451,
   -1,
   17242,
   451,
   -1,
   17242,
   17242,
   451,
   4229,
   451,
   17242,
   -1,
   451,
   17242,
   17242,
   451,
   16549,
   -1,
   -1,
   451,
   -1,
   18627,
   451,
   -1,
   21082,
   451,
   -1,
   9715,
   17242,
   3949,
   -1,
   17242,
   16549,
   451,
   6901,
   451,
   2470,
   14731,
   -1,
   13846,
   20191,
   451,
   -1,
   451,
   451,
   3949]},
 'L0_H2': {'freq': 0.0, 'features': []},
 'L0_H3': {'freq': 0.4399999976158142,
  'features': [11470,
   11470,
   11470,
   933,
   2591

If you want to look at MLP 3, all you have to do is access it:

In [9]:
cp.circuit_hypergraph['MLP3']

{'freq': 0.25999999046325684,
 'features': [1324,
  1324,
  8175,
  1324,
  8175,
  1324,
  1324,
  1324,
  1324,
  1324,
  1324,
  20313,
  8175,
  8175,
  1324]}

And just repeat what we did above:

In [11]:
features = list(set(cp.circuit_hypergraph['MLP3']['features']))
transcoder = transcoders[3]
feature_scores = get_feature_scores(model, transcoder, owt_tokens_torch, features, batch_size=64)

SparseTranscoder


100%|██████████| 400/400 [03:03<00:00,  2.18it/s]


In [17]:
feature_idx = 0 # corresponding to 16513
example_html, examples_clean_text = display_top_k_activating_examples(model, feature_scores[:, 0, :], owt_tokens_torch, k=5, show_score=True)

There's a few other methods, but you probably don't need to bother with those.

In [18]:
_ = cp.unique_feature_array(visualize=True)